In [1]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np

In [2]:
def gt_fn(x):
    #print x
    return (torch.max(x[:,0], x[:,1]) > (x[:,2]) + x[:,3]).float()

In [3]:
gt_fn(torch.rand(10,4)*10)


 1
 0
 0
 1
 0
 1
 0
 0
 0
 1
[torch.FloatTensor of size 10]

In [4]:
edges_np = np.array([(0,3),(3,1),(1,2),(2,3)])
edges_np = np.concatenate([edges_np,edges_np[:,[1,0]]])

In [5]:
edges_np

array([[0, 3],
       [3, 1],
       [1, 2],
       [2, 3],
       [3, 0],
       [1, 3],
       [2, 1],
       [3, 2]])

In [6]:
edge_selector = np.array([np.where(edges_np[:,0] == i)[0] for i in range(4)])

In [7]:
edge_selector

array([array([0]), array([2, 5]), array([3, 6]), array([1, 4, 7])], dtype=object)

In [8]:
edges = torch.LongTensor(edges_np)

In [9]:
edges


    0     3
    3     1
    1     2
    2     3
    3     0
    1     3
    2     1
    3     2
[torch.LongTensor of size 8x2]

In [10]:
weights = Variable(torch.rand(edges_np.shape), requires_grad=True)

In [11]:
weights

Variable containing:
 0.1689  0.5389
 0.5013  0.7015
 0.3318  0.5971
 0.2627  0.3875
 0.2855  0.4207
 0.5638  0.9463
 0.9046  0.8433
 0.3500  0.0762
[torch.FloatTensor of size 8x2]

In [12]:
# tocompute = torch.index_select(x, 1, Variable(edges.view(-1))).view(2,-1, 2)
# tocompute

In [13]:
optimizer = optim.SGD([weights], lr = 0.001, momentum=0.9)

In [22]:
batch_size = 2
torch.manual_seed(0)
x = Variable(torch.rand(batch_size,4)*10)
#print x
def GraphConv(x, edges):
    tocompute = torch.index_select(x, 1, Variable(edges.view(-1))).view(batch_size, -1, 2)
    #print tocompute
    conv = tocompute*weights
    #print conv
    for i, edges_to_select in enumerate(edge_selector):
        #print "x", conv
        #print "e", edges_to_select
        selected_edges = torch.index_select(conv, 1, Variable(torch.LongTensor(edges_to_select)))
        #print "m", selected_edges
        selected_edges = selected_edges.view(-1,edges_to_select.shape[0]*2)
        #print "m", selected_edges
        pooled_edges = torch.max(selected_edges,1)[0]
        #print "mmo",pooled_edges
        x[:,i] = pooled_edges
        #print "xx",x[:,i]
    return x

for i in range(2):
    x = GraphConv(x, edges)
    print "x",x
    
output = torch.nn.Sigmoid()(torch.nn.Linear(4,1)(x)).view(-1)

loss = torch.abs(Variable(gt_fn(x.data))-output)
print  loss

x Variable containing:
 4.5635  7.9537  6.4698  4.2387
 4.5796  7.9818  7.1936  6.0397
[torch.FloatTensor of size 2x4]

x Variable containing:
 2.2911  4.4676  6.6689  5.5992
 3.2646  5.6899  6.6925  5.6190
[torch.FloatTensor of size 2x4]

Variable containing:
 0.6548
 0.6722
[torch.FloatTensor of size 2]



In [23]:
loss.mean().backward()
optimizer.step()